# Αναγνώριση Προτύπων Μηχανικής Μάθησης
Εργασία 2 \\
Χρήστος Μπριστογιάννης sdi1900129 \\

##Ερώτημα 1: Αναγνώριση Προσώπων (Face recognition)

###Ζητούμενα i
Να γράψετε μία συνάρτηση loadImages(path, set_number) η οποία παίρνει ως είσοδο το
path στο οποίο βρίσκεται ο φάκελος των εικόνων π.χ. loadImages(“C:/images”, “Set_1”),
διαβάζει τις εικόνες και επιστέφει έναν πίνακα δεδομένων ανάλογα με το set_number,
όπου κάθε εικόνα αναπαρίσταται ως διάνυσμα στήλη. Η συνάρτηση επιστέφει επίσης τις
κατηγορίες (labels) στις οποίες ανήκουν οι διαφορετικές εικόνες κωδικοποιημένες με
ακεραίους (π.χ. 0 για φωτογραφίες που ανήκουν στο person_0, 1 για τις φωτογραφίες
που ανήκουν στο person_1 κτλ).

In [ ]:
import cv2
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

def loadImages(path, set_number):
  if set_number == "Set_1":
    start = 1
    end = 7
  elif set_number == "Set_2":
    start = 8
    end = 19
  elif set_number == "Set_3":
    start = 20
    end = 31
  elif set_number == "Set_4":
    start = 32
    end = 45
  elif set_number == "Set_5":
    start = 46
    end = 64
  else:
    print("Error")
    return

  arr = []
  labels = []
  for i in range(1, 11, 1):
    for j in range(start, end+1, 1):
      if i == 10:
        mes = 'person10'+'_'
      else:
        mes = 'person0'+str(i)+'_'
      if j < 10:
        mes = mes+'0'+str(j)
      else:
        mes = mes + str(j)

      img = cv2.imread(path+ mes + '.png', 0)

      arr.append(np.reshape(img,-1))
      labels.append(i)
  return arr, labels


#load images
numberOfSets = 5
nmsset = ["Set_1", "Set_2", "Set_3", "Set_4", "Set_5"]
imgset = [None] * numberOfSets
lblset = [None] * numberOfSets
path = '/content/drive/My Drive/faces/'

for i in range(0, numberOfSets):
  imgset[i], lblset[i] = loadImages(path, nmsset[i])
  # print(imgset[i])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###Ζητούμενα ii
Να εκπαιδεύσετε την μέθοδο Eigenfaces με d = 9 και d = 30 χρησιμοποιώντας όλες τις
εικόνες στο Set_1 (70 εικόνες) και να αναγνωρίσετε τα πρόσωπα στα Set_1 έως Set_5.
Για κάθε Set και κάθε τιμή της διάστασης d να αναφέρετε την ακρίβεια ταξινόμησης. Για
το Set_1 αναμένουμε 100% ακρίβεια ταξινόμησης καθώς χρησιμοποιήθηκε για την
εκπαίδευση της μεθόδου Eigenfaces. Σχολιάστε την δυνατότητα γενίκευσης της μεθόδου
στα διαφορετικά Sets.

In [ ]:
from sklearn.decomposition import PCA


n_components = 9 ## how may componets to keep
pca = PCA(n_components=n_components, whiten=True).fit(imgset[4])

print ("\n--pca--\n",pca)


components = pca.transform(imgset[4]) ## tranform the data to the new axes system)
print ("components.shape",components.shape)

##Ερώτημα 2: Ταξινόμηση εικόνων χρησιμοποιώντας SVMs

In [2]:
from sklearn.datasets import fetch_openml

# Φόρτωσει δεδομένων
mnist = fetch_openml('mnist_784')

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
import numpy as np
import random

# Μετατρέπουμε τις εικόνες σε μορφή διανύσματος διαστάσεων 28 x 28
X = np.array(mnist.data)
Y = np.array(mnist.target)
X = X.reshape(-1, 28, 28)

# Κανονικοποιούμε τα δεδομένα στο διάστημα [0,1]
X = X / 255.0

# Η συνάρτηση δέχεται τα δεδομένα data labels και τον τρόπο που θέλει να χωρίστουν τα set μεταξύ τους
# ανακατεύει την λίστα ώστε να μην έχουμε πάντα τα ίδια set μετά τα διασπάει σύμφωνα με sizes που δέχεται
# και μετά επιστρέφει 6 λίστες data και labels
def split_shuffle_dataset(data, labels, train_size, val_size, test_size):
  
  # Ένωση ανακάτεμα και διάσπαση λιστών data και labels
  combined = list(zip(data, labels))  
  random.shuffle(combined)
  shuffled_data, shuffled_labels = zip(*combined)

  # Διάσπαση νέων λιστών σε κομμάτια
  trainData = np.array(shuffled_data[:train_size])
  trainLbl = np.array(shuffled_labels[:train_size])
  valData = np.array(shuffled_data[train_size:train_size + val_size])
  valLbl = np.array(shuffled_labels[train_size:train_size + val_size])
  testData = np.array(shuffled_data[train_size + val_size:])
  testLbl = np.array(shuffled_labels[train_size + val_size:])

  return trainData, trainLbl, valData, valLbl, testData, testLbl

# Έτοιμες λίστες
trainData, trainLbl, valData, valLbl, testData, testLbl = split_shuffle_dataset(X, Y, 10000, 5000, 5000)


In [4]:
from sklearn import svm
from sklearn.metrics import accuracy_score
import time

kernel_values = ['linear', 'rbf']
c_values = [ 0.1, 1, 10, 100]
gamma_values = [ 0.001, 0.01, 0.1, 1]
sum_execution = 0


for kernel in kernel_values:
    best_accuracy = 0
    best_c = None
    best_gamma = None
    
    print(f'Testing {kernel} kernel...')

    for c in c_values:
        for gamma in gamma_values:

            # SVM train
            svm_model = svm.SVC(kernel=kernel, C=c, gamma=gamma)

            start_time = time.time()
            svm_model.fit(trainData.reshape(trainData.shape[0], -1), trainLbl)
            execution_time = time.time() - start_time
            sum_execution += execution_time
            
            # Έλενχος SVM στο validation set
            val_predictions = svm_model.predict(valData.reshape(valData.shape[0], -1))
            val_accuracy = accuracy_score(valLbl, val_predictions)
            
            #print(f' for values C={c} and gamma={gamma} the accuracy is {val_accuracy} and execution time is {execution_time}')
            
            # Εύρεση καλύτερου accuracy υπερπαραμέτρων
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                best_c = c
                best_gamma = gamma

    # Έλενχος SVM στο  Validation set με τις καλύτερες παραμέτρους
    svm_model = svm.SVC(kernel=kernel, C=best_c, gamma=best_gamma)
    svm_model.fit(trainData.reshape(trainData.shape[0], -1), trainLbl)

    # Έλενχος SVM στο Test set με τις καλύτερες παραμέτρους
    test_predictions = svm_model.predict(testData.reshape(testData.shape[0], -1))
    test_accuracy = accuracy_score(testLbl, test_predictions)

    # Εκτύπωση αποτελέσμάτων και για τα δύο kernel
    print(f'Best hyperparameters for {kernel} kernel: C={best_c}, gamma={best_gamma} and average execution time {sum_execution/(len(c_values)*len(gamma_values))}')
    print(f'Validation accuracy: {best_accuracy:.4f}')
    print(f'Test accuracy: {test_accuracy:.4f}\n')
    sum_execution = 0


Testing linear kernel...
Best hyperparameters for linear kernel: C=0.1, gamma=0.001 and average execution time 6.147791147232056
Validation accuracy: 0.9286
Test accuracy: 0.9288

Testing rbf kernel...
Best hyperparameters for rbf kernel: C=100, gamma=0.01 and average execution time 39.545528411865234
Validation accuracy: 0.9660
Test accuracy: 0.9651



In [5]:
from sklearn.decomposition import PCA

# 3 διαφορετικές τιμές για τη διατηρούμενη διακύμανση
variance_ratios = [ 0.5, 0.8, 0.95]

# PCA και SVM για κάθε variance_ratio με τις καλύτερες παραμέτρους
for variance_ratio in variance_ratios:
    print(f'Testing PCA with variance ratio {variance_ratio}...')

    # PCA
    pca = PCA(n_components=variance_ratio)
    pca_trainData = pca.fit_transform(trainData.reshape(trainData.shape[0], -1))
    pca_valData = pca.transform(valData.reshape(valData.shape[0], -1))
    pca_testData = pca.transform(testData.reshape(testData.shape[0], -1))
    components = pca.n_components_

    # SVM με καλύτερες παραμέτρους
    svm_model = svm.SVC(kernel=kernel, C=best_c, gamma=best_gamma)
    
    start_time = time.time()
    svm_model.fit(pca_trainData, trainLbl)
    execution_time = time.time() - start_time

    # Έλενχος SVM στο Validation set μετά απο pca
    val_predictions = svm_model.predict(pca_valData)
    val_accuracy = accuracy_score(valLbl, val_predictions)

    # Έλενχος SVM στο Test set μετά απο pca
    test_predictions = svm_model.predict(pca_testData)
    test_accuracy = accuracy_score(testLbl, test_predictions)

    print(f'Number of components: {components}')
    print(f'Validation accuracy: {val_accuracy:.4f}')
    print(f'Test accuracy: {test_accuracy:.4f}')
    print(f'Execution time: {execution_time:.4f} seconds\n')


Testing PCA with variance ratio 0.5...
Number of components: 11
Validation accuracy: 0.9342
Test accuracy: 0.9304
Execution time: 1.1545 seconds

Testing PCA with variance ratio 0.8...
Number of components: 43
Validation accuracy: 0.9652
Test accuracy: 0.9662
Execution time: 1.3266 seconds

Testing PCA with variance ratio 0.95...
Number of components: 152
Validation accuracy: 0.9670
Test accuracy: 0.9653
Execution time: 2.7512 seconds

